In [ ]:
from typing import List
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker
import os

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
MAIN_DIR = "../../../../"

plot_dir = f"{MAIN_DIR}/plots/BiasTest/23Jul18Res"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
cards_dir = "23Jul17ResClipTFScale1"
samples = [
    "NMSSM_XToYHTo2W2BTo4Q2B_MX-1200_MY-190",
    "NMSSM_XToYHTo2W2BTo4Q2B_MX-2000_MY-125",
    "NMSSM_XToYHTo2W2BTo4Q2B_MX-3000_MY-250",
]

biases = [0.0, 0.15, 0.3, 1.0]
r_dict = {}

for sample in samples:
    r_dict[sample] = {}
    for bias in biases:
        file = uproot.concatenate(
            f"/uscms/home/rkansal/hhcombine/cards/biastests/{cards_dir}/{sample}/higgsCombinebias{bias}.FitDiagnostics.mH125.*.root"
        )

        r = np.array(file.limit)[::4]
        r_negerr = r - np.array(file.limit)[1::4]
        r_poserr = np.array(file.limit)[2::4] - r
        reldiff = r - bias
        reldiff[reldiff < 0] = (reldiff / r_poserr)[reldiff < 0]
        reldiff[reldiff > 0] = (reldiff / r_negerr)[reldiff > 0]

        r_dict[sample][bias] = {
            "r": r,
            "reldiff": reldiff,
        }

In [ ]:
from scipy import stats

xrange = 4
x = np.linspace(-xrange, xrange, 101)

for sample in samples:
    fig, ax = plt.subplots(len(biases), 1, figsize=(12, 40))

    for i, bias in enumerate(biases):
        r = r_dict[sample][bias]["r"]
        reldiff = r_dict[sample][bias]["reldiff"]
        reldiff = reldiff[(reldiff > -4) * (reldiff < 4)]

        mu, sigma = np.mean(reldiff), np.std(reldiff)
        ax[i].hist(reldiff, np.linspace(-xrange, xrange, 21), histtype="step")
        ax[i].plot(
            x,
            stats.norm.pdf(x, loc=mu, scale=sigma) * len(r) * xrange / 10,
            label=rf"$\mu = {mu:.2f}, \sigma = {sigma:.2f}$",
        )
        ax[i].set_xlabel(rf"$\frac{{\hat{{r}} - {bias}}}{{\Delta \hat r}}$")
        ax[i].set_ylabel("Number of toys")
        ax[i].set_title(f"r = {bias}")
        ax[i].legend()

        hep.cms.label(
            "Work in Progress",
            ax=ax[i],
            data=True,
            lumi=138,
            year=None,
        )

    plt.savefig(f"{plot_dir}/{sample}.pdf", bbox_inches="tight")
    plt.show()